In [1]:
#https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [2]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [3]:
environment_name = "CartPole-v0"

In [4]:
env = gym.make(environment_name, render_mode='human')

/Users/djsanta/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [5]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done,turnc ,info = env.step(action)
        score +=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:32.0
Episode:2 Score:11.0
Episode:3 Score:37.0
Episode:4 Score:25.0
Episode:5 Score:13.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [6]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

0

In [7]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 1.0333197e+00, -2.1832868e+38, -1.9882593e-01, -3.2154252e+38],
      dtype=float32)

# 3. Train an RL Model

In [8]:
log_path = os.path.join('Training','Logs')

In [9]:
log_path

'Training/Logs'

In [10]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)


Using cpu device


/Users/djsanta/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [11]:
model.learn(total_timesteps=20000)

2023-08-01 15:29:05.951019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Logging to Training/Logs/PPO_5
-----------------------------
| time/              |      |
|    fps             | 94   |
|    iterations      | 1    |
|    time_elapsed    | 21   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 162         |
|    iterations           | 2           |
|    time_elapsed         | 25          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008806389 |
|    clip_fraction        | 0.0971      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0125     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.35        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0143     |
|    value_loss           | 50.9        |
-----------------------------------------
---

# 4. Save and Reload Model

In [12]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [13]:
model.save(PPO_path)

In [14]:
del model

In [15]:
PPO_path

'Training/Saved Models/PPO_model'

In [16]:
model = PPO.load('./Training/Saved Models/PPO_model', env=env)

# 4. Evaluation

In [17]:
from stable_baselines3.common.evaluation import evaluate_policy

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/Users/djsanta/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
/Users/djsanta/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/vec_env/base_vec_env.py:234: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(200.0, 0.0)

In [19]:
env.close()

# 5. Test Model

In [21]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': False, 'terminal_observation': array([-1.8768374 , -0.38609174, -0.22171952, -0.53931504], dtype=float32)}]


In [22]:
env.close()

# 6. Viewing Logs in Tensorboard

In [23]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [43]:
!tensorboard --logdir={training_log_path}

2023-08-01 15:10:59.454982: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.13.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [25]:
env.step(action)

(array([[-0.03988787, -0.20417427, -0.04834329,  0.2618274 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

# 7. Adding a callback to the training Stage

In [26]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [27]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [28]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

/Users/djsanta/anaconda3/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/djsanta/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [29]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [30]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [31]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 1433 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 940          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0079755075 |
|    clip_fraction        | 0.0781       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.00924     |
|    learning_rate        | 0.0003       |
|    loss                 | 8.49         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0103      |
|    value_loss           | 56.2         |
----------------------------

In [32]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [33]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [34]:
env.close()

# 8. Changing Policies

In [35]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [36]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


/Users/djsanta/anaconda3/lib/python3.10/site-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [37]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 925  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 508         |
|    iterations           | 2           |
|    time_elapsed         | 8           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.013553489 |
|    clip_fraction        | 0.17        |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.00711     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.38        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0205     |
|    value_loss           | 21.1        |
-----------------------------------------
----------------------------------

# 9. Using an Alternate Algorithm

In [38]:
from stable_baselines3 import DQN

In [39]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [40]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training/Logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.949    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 339      |
|    time_elapsed     | 0        |
|    total_timesteps  | 107      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.924    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 470      |
|    time_elapsed     | 0        |
|    total_timesteps  | 160      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.883    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 671      |
|    time_elapsed     | 0        |
|    total_timesteps  | 247      |
----------------------------------
------------------------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 108      |
|    fps              | 2240     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2312     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 2294     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2426     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 2315     |
|    time_elapsed     | 1        |
|    total_timesteps  | 2492     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 216      |
|    fps              | 2902     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4649     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 2925     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4730     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 2959     |
|    time_elapsed     | 1        |
|    total_timesteps  | 4839     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 324      |
|    fps              | 2960     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7214     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 2956     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7300     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 2960     |
|    time_elapsed     | 2        |
|    total_timesteps  | 7381     |
----------------------------------
----------------------------------
| rollout/          

Eval num_timesteps=9520, episode_reward=9.40 +/- 0.49
Episode length: 9.40 +/- 0.49
----------------------------------
| eval/               |          |
|    mean_ep_length   | 9.4      |
|    mean_reward      | 9.4      |
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    total_timesteps  | 9520     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 432      |
|    fps              | 2945     |
|    time_elapsed     | 3        |
|    total_timesteps  | 9580     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 2946     |
|    time_elapsed     | 3        |
|    total_timesteps  | 9670     |
------

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 532      |
|    fps              | 2968     |
|    time_elapsed     | 3        |
|    total_timesteps  | 11726    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 536      |
|    fps              | 2957     |
|    time_elapsed     | 3        |
|    total_timesteps  | 11812    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 540      |
|    fps              | 2960     |
|    time_elapsed     | 4        |
|    total_timesteps  | 11898    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 640      |
|    fps              | 3072     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14078    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 644      |
|    fps              | 3082     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14196    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 648      |
|    fps              | 3083     |
|    time_elapsed     | 4        |
|    total_timesteps  | 14260    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 748      |
|    fps              | 3171     |
|    time_elapsed     | 5        |
|    total_timesteps  | 16424    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 752      |
|    fps              | 3176     |
|    time_elapsed     | 5        |
|    total_timesteps  | 16518    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 756      |
|    fps              | 3186     |
|    time_elapsed     | 5        |
|    total_timesteps  | 16704    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 856      |
|    fps              | 3283     |
|    time_elapsed     | 5        |
|    total_timesteps  | 18975    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 860      |
|    fps              | 3291     |
|    time_elapsed     | 5        |
|    total_timesteps  | 19100    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 864      |
|    fps              | 3297     |
|    time_elapsed     | 5        |
|    total_timesteps  | 19259    |
----------------------------------
----------------------------------
| rollout/          

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()